In [1]:
!jupyter nbconvert --to script agent1.ipynb --output agent1
!jupyter nbconvert --to script pack_existing_segs.ipynb --output pack_existing_segs

[NbConvertApp] Converting notebook agent1.ipynb to script
[NbConvertApp] Writing 10508 bytes to agent1.py


/home/behnamnia/.conda/envs/rl39/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/behnamnia/.conda/envs/rl39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from agent1 import *
from pack_existing_segs import *
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation
from transformers import SamModel, SamProcessor, pipeline
import gc
from utils import show_mask, show_masks_on_image
from PIL import Image
import requests
import pickle
from pathlib import Path

In [3]:
if __name__ == "__main__":
    dataset = COCODataset(root="../Data/COCO17", train=True)
    generator = pipeline("mask-generation", model="facebook/sam-vit-huge", device=device)
    for img in dataset:
        if Path(f"../Data/COCO17/train2017seg/{img[1]}.pkl").exists():
            continue
        print('Processing', img[1])
        image = transforms.ToPILImage()(img[0])

        with torch.no_grad():
            outputs = generator(image, points_per_batch=64)
        packed_outputs = pack_seg_out(outputs)

        # plt.imshow(image)
        # plt.show()
        # show_masks_on_image(image, outputs['masks'])
        # plt.show()
        with open(f"../Data/COCO17/train2017seg/{img[1]}.pkl", "wb") as f:
            pickle.dump(packed_outputs, f)

In [4]:
# processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-large-coco-panoptic")
# model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-coco-panoptic")

In [5]:
# for img in dataset:
#     image = transforms.ToPILImage()(img[0])
#
#     inputs = processor(images=image, return_tensors="pt")
#     with torch.no_grad():
#         outputs = model(**inputs)
#
#     class_queries_logits = outputs.class_queries_logits
#     masks_queries_logits = outputs.masks_queries_logits
#     result = processor.post_process_panoptic_segmentation(outputs, target_sizes=[img[0].shape[2:0:-1]])
#     plt.imshow(image)
#     plt.show()
#     plt.imshow(result[0]['segmentation'])
#     plt.show()